In [1]:
import random
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from utils import load_all_images_3channel, get_class_name

PIXELS = 200

In [2]:
# settings
plt.style.use('seaborn')
plt.show()
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.figsize"] = (8, 5)

In [3]:
x, y = load_all_images_3channel(
    classes=["sunny", "cloudy", "foggy", "rainy", "snowy"], pixels=PIXELS
)

 has processed 18039

In [4]:
# make them as numpy array
x = np.array(x)
y = np.array(y)

x.shape, y.shape

((18039, 200, 200, 3), (18039,))

In [5]:
# shuffle and split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.8, shuffle=True, random_state=777
)

y_train_labels = y_train.copy()
y_test_labels = y_test.copy()

# convert class vector into binary matrix
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train.shape, y_train.shape

((14431, 200, 200, 3), (14431, 5))

# Feature extraction
## CNN with 3-Color Channels

In [12]:
model = Sequential([
    # keras.Input(shape=(PIXELS, PIXELS, 3)),    
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(PIXELS, PIXELS, 3)),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(units=64, activation='relu'),
    Dense(units=5, activation='softmax'),
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 99, 99, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 46, 46, 64)        36928     
                                                                 
 flatten_3 (Flatten)         (None, 135424)           

In [13]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [14]:
history = model.fit(
    x_train, y_train, epochs=5, batch_size=128, validation_split=0.2
)


Epoch 1/5


2022-08-21 19:05:01.588255: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


91/91 [==============================] - ETA: 0s - loss: 1.5736 - accuracy: 0.3892

2022-08-21 19:06:17.093375: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


91/91 [==============================] - 84s 895ms/step - loss: 1.5736 - accuracy: 0.3892 - val_loss: 1.1494 - val_accuracy: 0.5573
Epoch 2/5
91/91 [==============================] - 69s 763ms/step - loss: 1.3285 - accuracy: 0.4066 - val_loss: 0.9284 - val_accuracy: 0.6429
Epoch 3/5
91/91 [==============================] - 70s 765ms/step - loss: 1.2951 - accuracy: 0.4216 - val_loss: 0.9713 - val_accuracy: 0.6366
Epoch 4/5
91/91 [==============================] - 70s 767ms/step - loss: 1.2608 - accuracy: 0.4410 - val_loss: 1.1813 - val_accuracy: 0.5968
Epoch 5/5
91/91 [==============================] - 70s 768ms/step - loss: 1.2138 - accuracy: 0.4611 - val_loss: 1.1888 - val_accuracy: 0.5823


In [1]:
model.evaluate(x_test, y_test)

NameError: name 'model' is not defined